# Notebook to compare different runs varying coupling constants
## Gather Plaquette, polyakov loop and Acceptance 
Author: Venkitesh Ayyar (vayyar@bu.edu) \
May 4, 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sqlite3

import subprocess as sp
import os 
import glob
import pandas as pd
import itertools
import yaml
import sys

In [3]:
import gvar as gv

In [4]:
# %matplotlib widget

In [5]:
%matplotlib notebook

In [6]:
sys.path.append('modules')
from modules_parse_hmc_Grid import *
from modules_measurement import *


$$ \chi_\mathcal{O} = L^3 \left[ \langle \mathcal{O}^2 \rangle - {(\langle \mathcal{O} \rangle )}^2 \right] = L^3 \langle (\mathcal{O} - \bar{\mathcal{O}})^2\rangle $$ 

$$ \kappa_\mathcal{O} = \frac{\langle \mathcal{O}^4 \rangle - 4 \langle \mathcal{O}^3 \rangle \langle \mathcal{O} \rangle +6 \langle \mathcal{O}^2 \rangle {\langle \mathcal{O} \rangle} ^2 - 3 {\langle \mathcal{O} \rangle}^4 }{\chi_\mathcal{O}^2} = \frac{ \langle \left( \mathcal{O-\bar{\mathcal{O}}} \right) ^4 \rangle }{\chi_\mathcal{O}^2}$$ 

In [7]:
def f_get_run_info_dict(dict1,input_dict,run_label):
    '''   
    Get dictionary with run info that will be added to Dataframe as columns
    '''
    keys=['Lx','Lt','beta','mf']
    for key in keys:
        dict1[key]=input_dict[key]
    
    run_key='beta-%s_mf-%s_Lx-%s_Lt-%s'%(input_dict['beta'],input_dict['mf'],input_dict['Lx'],input_dict['Lt'])
    
    if input_dict['F_action'] =='Mobius_dwf': ## Add Ls in label for DWF
        keys.append('dwf_Ls')
    
        
    run_key='beta-%s_mf-%s_Lx-%s_Lt-%s'%(input_dict['beta'],input_dict['mf'],input_dict['Lx'],input_dict['Lt'])
            
    if input_dict['F_action'] =='Mobius_dwf': ## Add Ls in label for DWF
        run_key+='_Ls-%s'%(input_dict['dwf_Ls'])

    dict1['run_label'] = run_label
    run_key+='_'+run_label 
    
    dict1['run_key']   = run_key
        
    return dict1

In [8]:
def f_write_hmc_details_to_dbase(conn,Lx,Lt,top_dir,run_label):
    '''
    Combine results from different run extensions for a single run
    
    '''
    
    fldr_list=glob.glob(top_dir+'run_Lx-%s_Lt-%s*'%(Lx,Lt))

    for run_fldr in fldr_list: 
        print(run_fldr)
        # Read config file from folder
        input_dict=f_read_config(run_fldr+'/config.yaml')

        # Extract info from HMC output 
        flist=glob.glob(run_fldr+'/HSDM?.out')
        if len(flist)<1:    
            print("Not file HSDM*.out in %s"%(run_fldr))
            continue

        # sort output file list sequentially
        flist=[file_name.split('HSDM')[0]+'HSDM%s.out'%(i+1) for i,file_name in enumerate(flist)]

        ## Sum results from successive runs
        for idx,fname in enumerate(flist):

            try : 
                dict1={}
                df_a=f_parse_grid_data(fname)
                
                ## Add run data as columns of dataframe
                dict1=f_get_run_info_dict(dict1,input_dict,run_label)
                for key in dict1:
                    df_a[key]=dict1[key]
                                    
            except Exception as e:
                print(e,fname)
                continue

            if idx==0:
                print("idx",idx,fname)
                df=df_a.copy()
            else : 
                df=f_merge_df_successive_runs(df,df_a)
                
        # sqlite can't handle complex types, so convert to string
        df['Polyakov']=df['Polyakov'].astype(str)
        print(df.shape)
#         display(df)
        df.to_sql('test',conn,if_exists='append',index=False)
        
        
### Read sql database

def f_read_hmc_raw(fname):
    conn = sqlite3.connect(fname)
    df=pd.read_sql_query("SELECT * FROM test",conn)
    df['Polyakov']=df['Polyakov'].astype(np.complex128)
    conn.close()
    
    return df

## Write hmc data to sql database file

In [9]:

if __name__=="__main__" : 
    
    # Write data to sql database
    fname1='Stored_data/sql_dbase_files/parsed_hmc.db'
    
    if os.path.exists(fname1):
        os.remove(fname1)
    conn = sqlite3.connect(fname1)

    df_summary=pd.DataFrame([])
    top_dir='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/'
    f_write_hmc_details_to_dbase(conn,Lx='*',Lt='*',top_dir=top_dir,run_label='hot-start')

    top_dir='/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_4_cold_start/'
    f_write_hmc_details_to_dbase(conn,Lx='*',Lt='*',top_dir=top_dir,run_label='cold-start')
    conn.close()


/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.0_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.0_mf-0.1/HSDM1.out
(170, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.2_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.2_mf-0.1/HSDM1.out
(168, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.4_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.4_mf-0.1/HSDM1.out
(177, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_b

(60, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-24_Lt-48_Ls-16_beta-11.5_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-24_Lt-48_Ls-16_beta-11.5_mf-0.1/HSDM1.out
(63, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-11.5_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-11.5_mf-0.1/HSDM1.out
(1710, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-11.7_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-11.7_mf-0.1/HSDM1.out
(1210, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_

(286, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.9_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.9_mf-0.1/HSDM1.out
(1670, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-12.5_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-12.5_mf-0.1/HSDM1.out
(1411, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.7_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_5_hot_start/run_Lx-16_Lt-8_Ls-16_beta-10.7_mf-0.1/HSDM1.out
(1156, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_4_cold_start/run_Lx-1

(192, 11)
/usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_4_cold_start/run_Lx-24_Lt-12_Ls-16_beta-14.0_mf-0.1
idx 0 /usr/workspace/lsd/ayyar1/projects/SU4_sdm/runs_Grid/2023_july12/runs/phase_diagram_4_cold_start/run_Lx-24_Lt-12_Ls-16_beta-14.0_mf-0.1/HSDM1.out
(192, 11)


## Write summary data to sql dbase

In [10]:

if __name__=="__main__" : 
    
    ### Read in hmc data
    fname1='Stored_data/sql_dbase_files/parsed_hmc.db'
    df_read = f_read_hmc_raw(fname1)
    
    
    ### Create summary Dataframe
    df_summary=pd.DataFrame([])
    run_keys=np.unique(df_read.run_key.values)

    for rk in run_keys: # iterate over each ensemble
        df=df_read[df_read.run_key==rk].reset_index()

        equil = max(80,int(df.shape[0]*0.5))
        dict2={}
        for key in ['beta','mf','Lx','Lt','run_key','run_label']:
            dict2[key] = np.unique(df[key].values)[0]

        dict2=f_get_summary_data(df,dict2['Lx'],dict2,equil)
        df_summary=pd.concat([df_summary,pd.DataFrame(dict2,index=[df_summary.shape[0]+1])])  

    # Drop rows with no values
    df_summary=df_summary.sort_values(by=['beta','mf']).reset_index(drop=True)

    # Traj l = 2, so num_conf is twice the value
    df_summary['num_conf']=df_summary['num_conf']*2
    
    
    ### Write summary df to sql dbase
    # Convert gvars to strings for storing in Dataframe
    gvar_key_list=['plaq', 'sus_plaq', 'kurt_plaq', 'polyakov', 'sus_poly', 'kurt_poly', 'traj_time']
    for key in gvar_key_list:     df_summary[key]=df_summary[key].astype(str)
    
    fname2='Stored_data/sql_dbase_files/summary_hmc_data.db'
    
    if os.path.exists(fname2):
        os.remove(fname2)
    
    conn = sqlite3.connect(fname2)
    df_summary.to_sql('summary',conn,if_exists='replace',index=False)
    conn.close()
    